<a href="https://colab.research.google.com/github/MahdiTheGreat/Diagnostic-Systems/blob/main/DAIS_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MahdiTheGreat/Diagnostic-Systems.git
%cd Diagnostic-Systems
%ls
!pip install optuna

Cloning into 'Diagnostic-Systems'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 89.01 KiB | 1.56 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/Diagnostic-Systems
DAIS_4.ipynb  README.md  wdbc.pkl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.4 MB/s eta 0:00:00


In [2]:
import pickle
import pandas as pd
import numpy as np


# Load the pickle file
df = pd.read_pickle('wdbc.pkl')
df.drop('id', axis=1, inplace=True)

# Display the DataFrame
display(df[0:10])
display(df.columns)

# Basic statistics
display(df['malignant'].describe())

# Value counts (for categorical data)
display(df['malignant'].value_counts())


,malignant,radius_0,texture_0,perimeter_0,area_0,smoothness_0,compactness_0,concavity_0,concave points_0,symmetry_0,...,radius_2,texture_2,perimeter_2,area_2,smoothness_2,compactness_2,concavity_2,concave points_2,symmetry_2,fractal dimension_2
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,1,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,1,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,1,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,1,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


Index(['malignant', 'radius_0', 'texture_0', 'perimeter_0', 'area_0',
       'smoothness_0', 'compactness_0', 'concavity_0', 'concave points_0',
       'symmetry_0', 'fractal dimension_0', 'radius_1', 'texture_1',
       'perimeter_1', 'area_1', 'smoothness_1', 'compactness_1', 'concavity_1',
       'concave points_1', 'symmetry_1', 'fractal dimension_1', 'radius_2',
       'texture_2', 'perimeter_2', 'area_2', 'smoothness_2', 'compactness_2',
       'concavity_2', 'concave points_2', 'symmetry_2', 'fractal dimension_2'],
      dtype='object')

,malignant
count,569.000000
mean,0.372583
std,0.483918
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


,count
malignant,
0,357
1,212


In [21]:
from sklearn.model_selection import train_test_split
target_column = 'malignant'
X = df.drop(target_column, axis=1)  # Adjust 'target' to your actual target column name
y = df[target_column]  # Adjust 'target' to your actual target column name

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_, X_val, y_test_, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [22]:
from sklearn.tree import DecisionTreeClassifier,export_text,plot_tree
from sklearn.metrics import f1_score

tree=DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
print(f"f1 score: {f1_score(y_test, tree.predict(X_test)):.2f}")

f1 score: 0.93


In [23]:
import graphviz
# Print rules
tree_rules = export_text(tree, feature_names=X.columns)
print(tree_rules)
# Generate Graphviz format tree
dot_data = export_graphviz(tree,
                           feature_names=X.columns,
                           class_names=['benign',target_column],
                           filled=True, rounded=True, special_characters=True)

# Render and visualize the tree
graph = graphviz.Source(dot_data)
graph.render("decision_tree")  # Saves the tree as a file
graph.view()  # Opens the tree visualization

|--- concave points_0 <= 0.05
|   |--- radius_2 <= 16.83
|   |   |--- area_1 <= 48.70
|   |   |   |--- smoothness_2 <= 0.18
|   |   |   |   |--- smoothness_1 <= 0.00
|   |   |   |   |   |--- texture_2 <= 27.76
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- texture_2 >  27.76
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- smoothness_1 >  0.00
|   |   |   |   |   |--- texture_2 <= 33.35
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- texture_2 >  33.35
|   |   |   |   |   |   |--- texture_2 <= 33.56
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- texture_2 >  33.56
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |--- smoothness_2 >  0.18
|   |   |   |   |--- class: 1
|   |   |--- area_1 >  48.70
|   |   |   |--- concavity_1 <= 0.02
|   |   |   |   |--- class: 1
|   |   |   |--- concavity_1 >  0.02
|   |   |   |   |--- class: 0
|   |--- radius_2 >  16.83
|   |   |--- texture_0 <= 16.19
|   |   |   |--- class: 0
|   | 

'decision_tree.pdf'

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

def classify_cells_rule_based(df):
    # Define threshold values for abnormalities
    SIZE_THRESHOLD = 20
    AREA_THRESHOLD = 800
    PERIMETER_THRESHOLD = 90

    SHAPE_THRESHOLD = 0.25
    SMOOTHNESS_THRESHOLD = 0.15
    CONCAVITY_THRESHOLD = 0.2
    CONCAVE_POINTS_THRESHOLD = 5

    TEXTURE_THRESHOLD = 50
    SYMMETRY_THRESHOLD = 0.2
    FRACTAL_DIM_THRESHOLD = 1.2

    # Create a function to classify based on row conditions
    def classify_row(row):
        for i in range(3):  # Loop through the 3 feature sets (0, 1, 2)
            # Get the features for this set
            nuclear_radius = row[f'radius_{i}']
            nuclear_area = row[f'area_{i}']
            nuclear_perimeter = row[f'perimeter_{i}']
            compactness = row[f'compactness_{i}']
            smoothness = row[f'smoothness_{i}']
            concavity = row[f'concavity_{i}']
            concave_points = row[f'concave points_{i}']
            symmetry = row[f'symmetry_{i}']
            fractal_dimension = row[f'fractal dimension_{i}']
            texture = row[f'texture_{i}']

            # Rule-Based Classification
            if (nuclear_radius > SIZE_THRESHOLD or
                nuclear_area > AREA_THRESHOLD or
                nuclear_perimeter > PERIMETER_THRESHOLD):
                return 1
            elif (compactness > SHAPE_THRESHOLD or
                  smoothness > SMOOTHNESS_THRESHOLD or
                  concavity > CONCAVITY_THRESHOLD or
                  concave_points > CONCAVE_POINTS_THRESHOLD):
                return 1
            elif texture > TEXTURE_THRESHOLD:
                return 1
            elif (symmetry > SYMMETRY_THRESHOLD or
                  fractal_dimension > FRACTAL_DIM_THRESHOLD):
                return 1

        return 0

    # Apply the classification function row-wise
    y_pred = df.apply(classify_row, axis=1)
    return y_pred

# Example usage:
# Assuming 'df' is your DataFrame containing the features
# Split the dataset into features (X) and target (y)

# Get predictions on the test set
y_pred = classify_cells_rule_based(X)

# Evaluate the classifier
accuracy = accuracy_score(y, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.39015817223198596
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.03      0.06       357
           1       0.38      1.00      0.55       212

    accuracy                           0.39       569
   macro avg       0.65      0.51      0.30       569
weighted avg       0.72      0.39      0.24       569



In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import optuna


def objective(trial):
    # Define the hyperparameters to tune via Optuna
    n_estimators = trial.suggest_int('n_estimators', 50, 400)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 50)
    max_features = trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])

    # Create the random forest classifier
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # Fit the model
    clf.fit(X_train, y_train)

    # Evaluate the model
    y_pred = clf.predict(X_val)
    f1_score_value = f1_score(y_val, y_pred)

    return f1_score_value

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best f1_score_value:', study.best_trial.value)

# Best model parameters
best_params = study.best_trial.params
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
score = f1_score(y_test, y_pred)
print(f"f1 score: {score:.2f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import optuna

# Fit models and evaluate
log_reg=LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
print(f"Logistic Regression f1 score: {f1_score(y_test, log_reg.predict(X_test)):.2f}")

In [ ]:
model=SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score = f1_score(y_test, y_pred)
print(f"{name} f1 score: {score:.2f}")
